<a href="https://colab.research.google.com/github/geri-m/word2vec/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

The Goal of this sample to so tag (= classify) word types of a given text. The model should be able to answer for example the following questions as accuarte as possible. 
```
"Is the word 'will' in the sentense 'This is my last will' a noun?"
"Is the word 'will' in the sentense 'I will do that' a verb?"
```

## Background Information

The tags will be applied according to [conll2000](https://www.clips.uantwerpen.be/conll2000/chunking/), (Conference on Computational Natural Language Learning).

## Demonstration

The demo will show that by adding more context to a word (= taking surounding words into consideration), it is more likly to classify the word correctly.

# Prerequists

At first we make sure, Tensorflow is installed.

In [0]:
%tensorflow_version 2.x

import tensorflow as tf

print("TensorFlow Version: %s" % tf.__version__)
print("GPUs available: %s " % tf.config.list_physical_devices('GPU'))

In [0]:
# https://blog.cambridgespark.com/tutorial-build-your-own-embedding-and-use-it-in-a-neural-network-e9cde4a81296
import multiprocessing

import nltk
from gensim.models import Word2Vec
from nltk.corpus import brown

nltk.download('brown')
nltk.download('conll2000')

# Data is processed and Tokenised!
sentences = brown.sents()
print(sentences[:3])

EMB_DIM = 300

w2v = Word2Vec(sentences, size=EMB_DIM, window=5, min_count=5, negative=15, iter=10,
               workers=multiprocessing.cpu_count())

word_vectors = w2v.wv  # get trained embeddings - an KeyedVector instaces
result = word_vectors.similar_by_word("Saturday")
print("Most Similar to 'Saturday': %s" % result[:3])

In [0]:
from itertools import islice

for index, item in islice(word_vectors.vocab.items(), 10):
  print("Word: '%s'\tIndex in Dict: %s\tTotal Count in Input Text: %s" % (index, item.index, item.count))

In [0]:
from nltk.corpus import conll2000
from gensim.models import Word2Vec # https://code.google.com/archive/p/word2vec/
from tensorflow.keras.layers import Dense, Embedding, Activation, Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np
import collections

train_words = conll2000.tagged_words('train.txt')
test_words = conll2000.tagged_words('test.txt')
print(train_words[:20])
print(test_words[:20])
print("Amount of Trained Word-Tuple: %s" % len(train_words))
print("Amount of Test Word-Tuple: %s" % len(test_words))

In [0]:
from typing import Tuple
from typing import List
from typing import Dict
from typing import NoReturn

def get_tag_vocabulary(tagged_words: List[Tuple[str, str]]) -> Dict[str, int]:
  """
  Accepts text in the form of (word, pos) tuples and returns
  a dictionary mapping POS-tags to unique ids
  """
  tag2id = {}
  for item in tagged_words:
    tag = item[1]
    tag2id.setdefault(tag, len(tag2id))
  return tag2id

# the word_vectors.vocab dictionary stores Vocab objects, rather than integers
# but we would like our dictionary to map words to ints
# the word vector is some text, we are going to analyse
word2id = {k: v.index for k, v in word_vectors.vocab.items()}
# Result:  {'The': 14, 'Fulton': 5615, 'County': 1280, 'Grand': 5377, 'said': 59, 'Friday': 1852, 'an': 34, ...
# tag-2-id is from the trained data set. (we don't have tags yet in sentenses from word2vec)
tag2id = get_tag_vocabulary(train_words) 
# Result: {'NN': 0, 'IN': 1, 'DT': 2, 'VBZ': 3, 'RB': 4, 'VBN': 5, 'TO': 6, 'VB': 7, 'JJ': 8, 'NNS': 9, 'NNP': 10, ',': 11, 'CC': 12, 'POS': 13, '.': 14, 'VBP': 15, 'VBG': 16, 'PRP$': 17, 'CD': 18, '``': 19, "''": 20, 'VBD': 21, 'EX': 22, 'MD': 23, '#': 24, '(': 25, '$': 26, ')': 27, 'NNPS': 28, 'PRP': 29, 'JJS': 30, 'WP': 31, 'RBR': 32, 'JJR': 33, 'WDT': 34, 'WRB': 35, 'RBS': 36, 'PDT': 37, 'RP': 38, ':': 39, 'FW': 40, 'WP$': 41, 'SYM': 42, 'UH': 43}

In [0]:
UNK_INDEX = 0 # it is generally common to associate UNK with index 0
UNK_TOKEN = "UNK"

def get_int_data(tagged_words:List[str], word2id: Dict[str, int], tag2id:Dict[str, int]) -> Tuple[List[int], List[int]]:
  """
  Replaces all words and tags with their corresponding ids and
  separates words (features) from the tags (labels). 
  """

  X, Y = [], [] # X will hold word ids, Y will hold ids of their tags
  unk_count = 0 # to keep track of the number of unkonwn words
                # - words we don't have a representation for

  # iterate over the list of tagged words 
  for word, tag in tagged_words:
    # all possible tags for these tagged_words are already in tag2id
    # so get the ID an put it into Y
    Y.append(tag2id.get(tag))
    # check if the corresponding word is in the data we want to analyse
    if word in word2id:
      # if yes add to X (only the INDEX!)
      X.append(word2id.get(word))
    else:
      # if not, create an unkonwn word. (we only put INDEXs there)
      X.append(UNK_INDEX) # <---- NEW ADDED!
      unk_count += 1
  print("Data Created. percentag of unkown words: %.3f" % (unk_count/len(tagged_words)))
  return np.array(X), np.array(Y)

X_train, Y_train = get_int_data(train_words, word2id, tag2id)
X_test, Y_test = get_int_data(test_words, word2id, tag2id)

print("Result Data Len; X: %s, Y: %s" %(len(X_train), len(Y_train)))

# we need to one-hot encode the tag indexes
Y_train, Y_test = to_categorical(Y_train), to_categorical(Y_test)

In [0]:
def add_new_word(new_word: str, new_vector, new_index: int, embedding_matrix: np.ndarray, word2id: Dict[str, int]) -> Tuple[np.ndarray, List[Tuple[str, int]]]:
  """
  Adds a new word to the existing matrix of word embeddings.
  """
  # inserts the vector before given index, along axis 0
  embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)

  # updating the indexes of words that follow the new word
  word2id = {word: (index + 1) if index >= new_index else index 
             for word, index in word2id.items()}
  word2id[new_word] = new_index
  return embedding_matrix, word2id

# we add one single vector for the Unknown words
embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [0]:
HIDDEN_SIZE = 50
BATCH_SIZE = 128

def define_model(embedding_matrix: np.ndarray, class_count: int) -> Sequential:
  """
  Create and returns a simple part-of-speech model, which
  takes only one word as input
  """
  vocab_length = len(embedding_matrix)
  model = Sequential() # a sequential model is a stack of layers - we will add them one by one

  # Create and composing the model is the secret ingredint to Machine Learning
  # A layer which turns word indexes into vectors
  model.add(Embedding(input_dim = vocab_length,
                      output_dim=EMB_DIM, # output of this layer is the embedding of the input word
                      weights=[embedding_matrix], # the matrix holding the trained embeddings
                      input_length=1)) # specifies how many indexes we are looking up
  model.add(Flatten())
  model.add(Dense(HIDDEN_SIZE))
  model.add(Activation("tanh"))
  model.add(Dense(class_count))
  model.add(Activation("softmax"))

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="categorical_crossentropy",
                metrics=["accuracy"])
  return model

pos_model = define_model(embedding_matrix, len(tag2id))
pos_model.summary()

# Training the model. 
# Input Parameter for the model are words of a tokenzied senetense (Indexes only)
# Output Parameter of the model are types/tags of the words
pos_model.fit(X_train,
              Y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              verbose=1)



In [0]:
def evaluate_model(model : Sequential, id2word:List[str], x_test: List[int], y_test: List[int]):
  """
  Evaluates the given model by computing the accuracy of its predictions
  on the given test data and prints out 10 most mistagged words.
  """
  _, acc = model.evaluate(x_test, y_test) # get accuracy of the model
  print("Accuracy: %.2f" % acc)


  # the following lines are used to get most commonly misstagged words
  y_pred = model.predict_classes(x_test) # get model predictions
  error_counter = collections.Counter()  # we will use a counter instance to count model's erros

  for i in range(len(x_test)):
    correct_tag_id = np.argmax(y_test[i]) # turn a one-hot encoding to an index
    if y_pred[i] != correct_tag_id:
      word = id2word[x_test[i]]
      error_counter[word] += 1

  print("Most commen errors:\n", error_counter.most_common(10))

# List of all words sorted by their ID
id2word = sorted(word2id, key=word2id.get)
evaluate_model(pos_model, id2word, X_test, Y_test)


In [0]:
EOS_INDEX = 1
EOS_TOKEN = "EOS"

# creating a random end-of-sequence vector
eos_vector = np.random.standard_normal(EMB_DIM)
embedding_matrix, word2id = add_new_word(EOS_TOKEN, eos_vector, EOS_INDEX, embedding_matrix, word2id)

In [0]:
CONTEXT_SIZE = 2 # define the size of the context-window


def get_window_int_data(tagged_words: List[Tupel[str, str]], word2id: Dict[str, int], tag2id: Dict[str, int]) -> Tuple[List[List[int]]]:
  """
  Replaces all words and tags with their corresponding ids and
  generates an array of label ids Y and the traing data X which
  consists of arrays of word indexes (of tagged word and its context).
  """

  # create list-of-list
  X, Y = [],[]
  unk_count = 0

  span = 2 * CONTEXT_SIZE + 1 # the complete span of the sliding window -> [ window traget window]
  buffer = collections.deque(maxlen=span)
  padding = [(EOS_TOKEN, None)] * CONTEXT_SIZE
  buffer += padding + tagged_words[:CONTEXT_SIZE]

  for item in (tagged_words[CONTEXT_SIZE:] + padding):
    buffer.append(item)

    # the input to the model is the ids of all words in the window
    window_ids = np.array([word2id.get(word) if (word in word2id) else UNK_INDEX for (word, _) in buffer])
    X.append(window_ids)

    # the label is the tag of the middle word
    middle_word, middle_tag = buffer[CONTEXT_SIZE]
    Y.append(tag2id.get(middle_tag))

    if middle_word not in word2id:
      unk_count += 1

  print("Data Created, Percentage of unknown words: %.3f" % (unk_count/len(tagged_words)))
  return np.array(X), np.array(Y)

In [0]:
def define_context_sensitive_model(embedding_matrix: np.ndarray, class_count: int):
  """
  Create and returns a part-of-speech model, which
  takes as input a tagged word and its context.
  """

  vocab_length = len(embedding_matrix)
  total_span = CONTEXT_SIZE * 2 +1 

  model = Sequential()
  model.add(Embedding(input_dim=vocab_length,
                       output_dim=EMB_DIM,
                       weights=[embedding_matrix],
                       input_length=total_span)), # <----

  model.add(Flatten())
  model.add(Dense(HIDDEN_SIZE))
  model.add(Activation("tanh"))
  model.add(Dense(class_count))
  model.add(Activation("softmax"))

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="categorical_crossentropy",
                metrics=["accuracy"])
  return model



In [0]:
def evaluate_model_ext(model: Sequential, id2word:List[int], x_test:List[int], y_test:List[int]):
  """
  Evaluates the given model by computing the accuracy of its predictions
  on the given test data and prints out 10 most mistagged words.
  """
  _, acc = model.evaluate(x_test, y_test) # get accuracy of the model
  print("Accuracy: %.2f" % acc)


  # the following lines are used to get most commonly misstagged words
  y_pred = model.predict_classes(x_test) # get model predictions
  error_counter = collections.Counter()  # we will use a counter instance to count model's erros

  for i in range(len(x_test)):
    correct_tag_id = np.argmax(y_test[i]) # turn a one-hot encoding to an index
    if y_pred[i] != correct_tag_id:       # sic! y_pred <=> y_new
      if isinstance(x_test[i], np.ndarray): 
        word = id2word[x_test[i][CONTEXT_SIZE]]
      else:
        word = id2word[x_test[i]]
      error_counter[word] += 1
  
  print("Most commen errors:\n", error_counter.most_common(10))

In [0]:
X_train2, Y_train2 = get_window_int_data(train_words, word2id, tag2id)
X_test2, Y_test2 = get_window_int_data(test_words, word2id, tag2id)
Y_train2, Y_test2 = to_categorical(Y_train2), to_categorical(Y_test2)

cs_pos_model = define_context_sensitive_model(embedding_matrix, len(tag2id))
cs_pos_model.fit(X_train2,
                 Y_train2,
                 batch_size=BATCH_SIZE,
                 epochs=1,
                 verbose=1)

evaluate_model_ext(cs_pos_model, id2word, X_test2, Y_test2)